Using VGG16 CNN architecture on the muffin vs chihuahua dataset

In [1]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

In [2]:
# DATASET DIRECTORY CONFIGURATION
train_dir = "muffin-vs-chihuahua/train"
test_dir = "muffin-vs-chihuahua/test"

In [3]:
# IMAGE PARAMETERS
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [4]:
# DATA PREPROCESSING & AUGMENTATION
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)


Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [5]:
# VGG16 MODEL
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model.trainable = False  # freeze convolutional base

# Build the classifier on top
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [6]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [7]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 117s 972ms/step - accuracy: 0.9153 - loss: 1.2643 - val_accuracy: 0.9704 - val_loss: 0.2796
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 74s 622ms/step - accuracy: 0.9652 - loss: 0.3410 - val_accuracy: 0.9757 - val_loss: 0.2421
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 74s 623ms/step - accuracy: 0.9718 - loss: 0.2211 - val_accuracy: 0.9704 - val_loss: 0.2845
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 79s 661ms/step - accuracy: 0.9762 - loss: 0.1268 - val_accuracy: 0.9767 - val_loss: 0.1863
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 76s 641ms/step - accuracy: 0.9791 - loss: 0.1035 - val_accuracy: 0.9810 - val_loss: 0.1236
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 77s 646ms/step - accuracy: 0.9831 - loss: 0.0768 - val_accuracy: 0.9714 - val_loss: 0.1895
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 76s 634ms/step - accuracy: 0.9865 - loss: 0.0742 - val_accuracy: 0.9788 - val_loss: 0.1729
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 69s 584ms/step - accuracy: 0.9868 - loss: 

In [8]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")


37/37 ━━━━━━━━━━━━━━━━━━━━ 25s 688ms/step - accuracy: 0.9823 - loss: 0.1082
Test Accuracy: 0.9822635054588318


In [9]:
# Save the model
model.save('exercise_6_vgg16_villarin.h5')